# RW - LM spec migration management v2 -> v4.

The aim of this script is to programatically allow us to move from the v2 layer spec from layer manager used in RW layers to the v4.

We will take as started point rw past migration and latest migration done by gfw. plus scripts develop for OW migration management from staging and productions


Tasks prior running the process:

* RW management functions
* Develop functions/code to move from spect v2 to v4
* Develop test to ensure functions works as expected

Finally the workflow will be excecuted one provider type at a time, and will be as follows:  

* Migrate to staging 5 layers of each provider
* For those 5 layers we will run the script to migrate and validate.
* Once Front is happy with the results we will trigger the full migration

In [3]:
### Run functions developed in the notebooks

%run './RW-API_management_functions.ipynb'
%run './RW-LM4_migration_functions.ipynb'
%run './pydantic_data_classes_rw.ipynb'

builtins==None
builtins==None
requests_toolbelt.sessions==None
types==None
json==2.0.9
logging==0.5.1.2
getpass==None


In [2]:
from devtools import debug

In [21]:
from IPython.display import JSON, display

In [5]:
session = rwApiSession()

You are login into RW api
Email: alicia.arenzana@vizzuality.com
Password: ········
Successfully logged into RW api


### Explore and get some metrics about the layers currently in RW

In [6]:
layerList = getLayerList(session)

In [25]:
display(f' currently available {len(layerList)} layers in the api')

 currently available 5427 layers in the api


In [8]:
display(layerList[0])
JSON(layerList[0].json(exclude_none=True))

**Layer**: [1993 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/b9d1b315-efc3-433a-82ad-22e841c34f5a)

In [33]:
providers = {}
for layer in layerList:
    if layer.attributes.provider in providers.keys():
        providers[layer.attributes.provider]['count'] += 1
        providers[layer.attributes.provider]['ids'].append(layer.id)
    else:
        providers[layer.attributes.provider] = {'count': 1, 'ids':[layer.id]} 

for k,v in providers.items():
    print(f'{k}: {v["count"]}')

cartodb: 3501
gee: 1591
leaflet: 280
featureservice: 19
wms: 35
None: 1


This means in our migration we will need to only take care of 5 type of connectors

#### Review the layer with None as provider and exclude it from our migration

In [38]:
for layer in layerList:
    if layer.attributes.provider is None:
        display(layer)
        display(JSON(layer._url))

**Layer**: [Brazilian Amazon Deforestation Alerts](https://api.resourcewatch.org/v1/dataset/8870d62f-0240-4275-910a-361f0ef347e0/layer/0c07f80c-129d-47b3-9099-99036883a215)

<IPython.core.display.JSON object>

#### Lets have a look at the layerType that indicates a successfull migration from v1 to v2

In [39]:
layerTypes = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('layerType') in layerTypes.keys():
        layerTypes[layer.attributes.layerConfig.get('layerType')] += 1
    else:
        layerTypes[layer.attributes.layerConfig.get('layerType')] = 1
layerTypes

{'vector': 2730, None: 1871, 'raster': 826}

#### From those that look unsusscessfull lets have a look at the providers, we will need to be extra carefull with this ones:

In [40]:
layer2Types = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('layerType') is None:
        if layer.attributes.provider in layer2Types.keys():
            layer2Types[layer.attributes.provider] += 1
        else:
            layer2Types[layer.attributes.provider] = 1
layer2Types

{'gee': 1432,
 'cartodb': 282,
 'featureservice': 18,
 'wms': 15,
 'leaflet': 123,
 None: 1}

#### Add some testing layers from prod to staging from each different type so we can have some testing data.

In [58]:
testing_layers = []
for k,v in providers.items():
    print(f'{k}: ')
    for ID in v["ids"][:5]:
        layer = parse_obj_as(Layer, session.get(f'v1/layer/{ID}').json().get('data'))
        display(layer)
        copyLayer = layer.copy(deep=True)
        copyLayer.attributes.env = 'staging'
        testing_layers.append(session.post(f'v1/dataset/{copyLayer.attributes._datasetId}/layer', data = copyLayer.attributes.json(exclude_none=True))\
                              .json().get('data').get('id'))

cartodb: 


**Layer**: [1993 Electricity Distribution Losses (kWh, billions)](https://api.resourcewatch.org/v1/dataset/a36bf77b-4d10-4855-8ef6-aece72e23971/layer/b9d1b315-efc3-433a-82ad-22e841c34f5a)

'staging'

'production'

#### Start migration on the testing layers

In [ ]:
createUpdatelayerSchemas(session, testing_layers)

#### Once we are happy with the layer we will proceed to migrate all +5000 of them:

In [ ]:
for k,v in providers:
    print(k)
    createUpdatelayerSchemas(session, v["ids"])

### Aside:
##### encode/decode vector tiles for testing data purpouses

In [49]:
import mapbox_vector_tile
import requests

In [ ]:
mapbox_vector_tile.encode([
    {
        "name": "errorTileSquareLayer",
        "features": [{ "geometry":"POLYGON ((0 0, 0 4096, 4096 4096, 4096 0, 0 0))" }]
    },
    {
        "name": "errorTileStripesLayer",
        "features": [{ "geometry":"POLYGON ((0 0, 0 4096, 4096 4096, 4096 0, 0 0))" }]
    }
])

In [51]:
response = requests.get('<url>')
mapbox_vector_tile.decode(response.content)

{'layer0': {'extent': 4096,
  'version': 2,
  'features': [{'geometry': {'type': 'Polygon',
     'coordinates': [[[3978, 3191],
       [4063, 3320],
       [4232, 3320],
       [4316, 3191],
       [4232, 3061],
       [4063, 3061],
       [3978, 3191]]]},
    'properties': {'puGeomId': '337a2b08-3fdc-46ea-9580-2ac5ee93b5c8',
     'scenarioPuId': '6424c619-6ccf-454b-ae50-bccf0c6645de',
     'puid': 27,
     'parseKeys': 'valuePosition,featureList',
     'costValue': 1.0},
    'id': 0,
    'type': 3},
   {'geometry': {'type': 'Polygon',
     'coordinates': [[[3978, 2930],
       [4063, 3061],
       [4232, 3061],
       [4316, 2930],
       [4232, 2800],
       [4063, 2800],
       [3978, 2930]]]},
    'properties': {'puGeomId': '12a227a2-e017-4335-afae-cfba9d381442',
     'scenarioPuId': '6f962395-48aa-492c-9ef8-d971b86259e2',
     'puid': 29,
     'parseKeys': 'valuePosition,featureList',
     'costValue': 1.0},
    'id': 0,
    'type': 3},
   {'geometry': {'type': 'Polygon',
     'co